# **ETL Report for the US Census Bureau's Annual Business Survey**

# Introduction

---

The US Census Bureau's Annual Business Survey (ABS) gathers demographic and economic information about businesses and business owners. There are four main ABS datasets aggregated by the Census Bureau and made available to the public via an API We will be looking at the 2019 survey results. According to the Census Bureau:<br>
"**Company Summary** provides data for employer businesses by sector, sex, ethnicity, race, veteran status, years in business, receipts size of firm, and employment size of firm for the U.S., states, and metro areas. "<br>
"**Characteristics of Businesses** provides data for respondent employer firms by sector, sex, ethnicity, race, veteran status, years in business, receipts size of firm, and employment size of firm for the U.S., states, and metro areas, including detailed business characteristics."<br>
"**Characteristics of Business Owners** provides data for owners of respondent employer firms by sector, sex, ethnicity, race, and veteran status for the U.S., states, and metro areas, including detailed owner characteristics."<br>
"**Technology Characteristics of Businesses** provides data on technology use and production for Artificial Intelligence, Cloud-Based Computing, Specialized Software, Robotics, and Specialized Equipment technologies data at the U.S and State level for 2018."

When looking at data at the state level, we wanted to normalize the data by population. We pulled in another CSV from the Census Bureau that Contained population data for each state.

In order to create map visualizations, we also needed to pull in a CSV that contained state names and abbreviations.

These data sets need to be transformed in order to make meaningful visualizations. 

# Data Sources

---

The documentation for the US Census Bureau's Annual Business Survey API can be found here https://www.census.gov/data/developers/data-sets/abs.2019.html and the data is accessed by calling the API.


The documentation for the US Census Bureau's Population Esitmations can be found here https://www.census.gov/newsroom/press-kits/2019/national-state-estimates.html and the data is accessed by downloading NST-EST2019-alldata CSV.

The CSV for the abbreviations can be accessed here https://worldpopulationreview.com/states/state-abbreviations.


### Citations
US Census Bureau. (2021, October 14). Annual Business Survey (ABS) APIs. Census.gov. Retrieved September 2, 2022, from https://www.census.gov/data/developers/data-sets/abs.2019.html

US Census Bureau. (2021, October 8). 2019 National and State Population Estimates. Census.gov. Retrieved September 2, 2022, from https://www.census.gov/newsroom/press-kits/2019/national-state-estimates.html

List of State Abbreviations (Download CSV, JSON). Retrieved September 2, 2022, from https://worldpopulationreview.com/states/state-abbreviations
# Transforming and Cleaning
---

## **Imports and API Key**

The following python libraries are necessary for transforming the data: `pandas`, `requests`. API keys can be obtained by following this link: https://www.census.gov/data/developers/guidance/api-user-guide.Help_&_Contact_Us.html and clicking on 'Request a Key' on the left-hand side of the page. The personal API key should be stored in a config.py file.


In [1]:
import pandas as pd
from Configuration.config_ import apikey
import requests

## **Census Population Data by State**
In order to later standardize the the aggregated census value data column 'FIRMPDEMP', we imported a seconday census dataset showing National, Regional, and State total populations to hopefully reduce population size bias. The data come from <href>https://www.census.gov/newsroom/press-kits/2019/national-state-estimates.html</href> which has total populations from a range of years with excess data meaning it required cleaning. 

1. Read in the data on the excel `'nst-est2019-01.xlsx'` table with the pandas `pd.read_excel()` function, skipping 3 rows to remove excess title information. 

In [3]:
census_national_populations = pd.read_excel("./Files/nst-est2019-01.xlsx", skiprows=3)

2. Rename to two needed columns in the data so that `'Unamed 0'` becomes `'State'` and `2018` becomes `'Population'`.

In [4]:
census_national_populations = census_national_populations.rename(columns={"Unnamed: 0": "State", 2018:"Population"})

3. Remove all other columns using the pandas `pd.drop()` function. 

In [5]:
census_national_populations = census_national_populations.drop(["Census", "Estimates Base",2010,2011,2012,2013,2014,2015,2016,2017,2019],axis=1)

4. Remove any unneeded row once again using the `pd.drop()` function. 

In [6]:
census_national_populations = census_national_populations.drop([0,1,2,3,4,56,57,58,59,60,61,62])

5. Use lambda functions to map all the `'State'` column data to be a string and all the `'Population'` column data to be a float. 
6. In the lambda function for the `'State'` column we also remove the first character from each string which was a period makes it so we can't accurately refrence the state location.

In [7]:
census_national_populations['State'] = census_national_populations['State'].apply(lambda x : str(x[1:]))
census_national_populations["Population"] = census_national_populations["Population"].apply(lambda x : float(x))

7. Reset the index, moving a secondary manipulated index column into the DataFrame, then remove it with `pd.drop()` as it is an excess column only refrencing the unmanipluated table previously imported. 

In [8]:
census_national_populations.reset_index(inplace=True)
census_national_populations = census_national_populations.drop("index",axis=1)

This leaves us with `census_national_populations` DataFrame, which contains state names and total population in 2018.

## **Company Summary**

### Calling the API
1. Request both U.S. and State data from the 2019 Annual Business Survey (ABS) Company Summary. We use `request.get()` to pull the data from the Census website then use requests method `.json()` in order to transform the data into a list of lists split by newlines at the end of each row. We only import columns variables `NAME`, `SEX`, `SEX_LABEL`, `RACE_GROUP`, `RACE_GROUP_LABEL`, `FIRMPDEMP`. This is to limit unneeded data calls and grab needed information for later visualizations.<br>

    For US Level Data
    https://api.census.gov/data/2018/abscs?get=NAME,SEX,SEX_LABEL,RACE_GROUP,RACE_GROUP_LABEL,FIRMPDEMP&for=us:*&key={apikey}

    For State Level Data
    https://api.census.gov/data/2018/abscs?get=NAME,SEX,SEX_LABEL,RACE_GROUP,RACE_GROUP_LABEL,FIRMPDEMP&for=state:*&key={apikey}
    
    The imported U.S. data is named `reponse_company_summary_Nation` and the state data is named `reponse_company_summary_State`.

In [9]:
reponse_company_summary_Nation = requests.get(f"https://api.census.gov/data/2018/abscs?get=NAME,SEX,SEX_LABEL,RACE_GROUP,RACE_GROUP_LABEL,FIRMPDEMP&for=us:*&key={apikey}").json()
reponse_company_summary_State = requests.get(f"https://api.census.gov/data/2018/abscs?get=NAME,SEX,SEX_LABEL,RACE_GROUP,RACE_GROUP_LABEL,FIRMPDEMP&for=state:*&key={apikey}").json()

2. Transform both datasets from list of lists to a pandas DataFrame using `pd.DataFrame()`. The first list of each dataset was set to 'columns' meaning it would be the header of the table and then set the rest of the dataset to 'data' representing each list as a row in the DataFrame. The list `reponse_company_summary_Nation` transforms to `df_company_summary_Nation`, and `reponse_company_summary_State` transforms to `df_company_summary_State`.

In [10]:
df_company_summary_Nation = pd.DataFrame(data=reponse_company_summary_Nation[1:], columns=reponse_company_summary_Nation[0])
df_company_summary_State = pd.DataFrame(data=reponse_company_summary_State[1:], columns=reponse_company_summary_State[0])

### Data Cleaning and Transforming

*State Level*
1. Make a copy of `df_company_summary_State` for ease of use.

In [11]:
df_company_summary_State = df_company_summary_State.copy()

2. Rename the column `'NAME'` in `df_company_summary_State` to `'State'`

In [12]:
df_company_summary_State = df_company_summary_State.rename(columns={"NAME":"State"})

3. Merge `df_company_summary_State` with `census_national_populations` with an inner merge on the column `'State'`.

In [13]:
df_company_summary_State = df_company_summary_State.merge(census_national_populations, how= 'inner', on="State")

4. Drop the `'state'` column as it is unessacary for later visualizations.

In [14]:
df_company_summary_State = df_company_summary_State.drop(["state"], axis = 1)

5. Rename the column `'Population'` to `'Total_Population'` so it's easier to understand the meaning of the column. 

In [15]:
df_company_summary_State = df_company_summary_State.rename(columns={"Population":"Total_Population"})

6. Set the columns `'FIRMPDEMP'` and `'Total_Population'` to integers to allow later amthimatical transformations.

In [16]:
df_company_summary_State["FIRMPDEMP"] = df_company_summary_State["FIRMPDEMP"].astype(int)
df_company_summary_State["Total_Population"] = df_company_summary_State["Total_Population"].astype(int)

7. Copy `df_company_summary_State` and setting it to `df_company_summary_State_bussniess_per_population` to better refrence the meaning of the dataset. 

In [17]:
df_company_summary_State_bussniess_per_population = df_company_summary_State.copy()

8. Create a new column called `'Bus_Num_by_Pop'` to refrence a column finding the number of busniesses based on the total population of the state which is calculated by dividing the `'FIRMPDEMP'` column by the `'Total_Population'` column that is then mutliplied by 100 to create a proportional percentage column. 

In [18]:
df_company_summary_State_bussniess_per_population["Bus_Num_by_Pop"] = (df_company_summary_State_bussniess_per_population["FIRMPDEMP"] / df_company_summary_State_bussniess_per_population["Total_Population"]) * 100

9. Remove unneeded columns; `'RACE_GROUP'`, `'Total_Population'`, `'SEX'`, `'FIRMPDEMP'`.

In [19]:
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population.drop(["RACE_GROUP", "Total_Population", "SEX", "FIRMPDEMP"], axis=1)

10. Clean the remaining columns by taking out rows with labels that aggregagte rows together such as `'Total'` and `'Equally male/female'` for the `'SEX_LABEL'` column and `'Total'`, `'Minority'`, `'Nonminority'`, and `'Equally minority/nonminority'` for the `'RACE_GROUP_LABEL'` column. This leaves `df_company_summary_State_bussniess_per_population` with only 4 columns.

In [20]:
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population[df_company_summary_State_bussniess_per_population["SEX_LABEL"] != "Total"]
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population[df_company_summary_State_bussniess_per_population["SEX_LABEL"] != "Equally male/female"]
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population[df_company_summary_State_bussniess_per_population["RACE_GROUP_LABEL"] != "Total"]
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population[df_company_summary_State_bussniess_per_population["RACE_GROUP_LABEL"] != "Minority"]
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population[df_company_summary_State_bussniess_per_population["RACE_GROUP_LABEL"] != "Nonminority"]
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population[df_company_summary_State_bussniess_per_population["RACE_GROUP_LABEL"] != "Equally minority/nonminority"]

11. Use a complex pandas `groupby()` function to group by categorical columns `'State'`, `'SEX_LABEL'`, and `'RACE_GROUP_LABEL'` then aggregate a sum of the `'Bus_Num_by_Pop'` by grouping. 

In [21]:
df_company_summary_State_bussniess_per_population.groupby(["State", "SEX_LABEL", "RACE_GROUP_LABEL" ] ,as_index=False)["Bus_Num_by_Pop"].sum()

,State,SEX_LABEL,RACE_GROUP_LABEL,Bus_Num_by_Pop
0,Alabama,Female,American Indian and Alaska Native,0.000000
1,Alabama,Female,Asian,0.030055
2,Alabama,Female,Black or African American,0.011007
3,Alabama,Female,Native Hawaiian and Other Pacific Islander,0.000000
4,Alabama,Female,White,0.209650
...,...,...,...,...
492,Wyoming,Male,American Indian and Alaska Native,0.000000
493,Wyoming,Male,Asian,0.036877
494,Wyoming,Male,Black or African American,0.000000
495,Wyoming,Male,Native Hawaiian and Other Pacific Islander,0.000000


12. Sort the DataFrame alphabeticly by categorical columns `'State'`, `'SEX_LABEL'`, and `'RACE_GROUP_LABEL'`. This gives us a sorted aggragated DataFrame which can be refrenced by state, sex, and race showing the numericaly standarized column `'Bus_Num_by_Pop'`.

In [22]:
df_company_summary_State_bussniess_per_population = df_company_summary_State_bussniess_per_population.sort_values(by=["State","RACE_GROUP_LABEL", "SEX_LABEL"], ascending=True)
df_company_summary_State_bussniess_per_population.head()

,State,SEX_LABEL,RACE_GROUP_LABEL,Bus_Num_by_Pop
950,Alabama,Female,American Indian and Alaska Native,0.000000
951,Alabama,Male,American Indian and Alaska Native,0.004910
954,Alabama,Female,Asian,0.030055
955,Alabama,Male,Asian,0.063752
942,Alabama,Female,Black or African American,0.011007


*US Level*
1. Copy the `df_company_summary_Nation` DataFrame and named the copy `df_company_summary_Overall` to show the data would be changed. 

In [23]:
df_company_summary_Overall = df_company_summary_Nation.copy()

2. Rename the `'NAME'` column `'Location'`.

In [24]:
df_company_summary_Overall = df_company_summary_Overall.rename(columns={"NAME":"Location"})

3. Drop the `'RACE_GROUP'`, `'SEX'`, and `'us'` columns

In [25]:
df_company_summary_Overall = df_company_summary_Overall.drop(["RACE_GROUP", "SEX", "us"], axis=1)

4. Remove unneeded columns in the `df_company_summary_Overall` DataFrame such as `'Total'` and `'Equally male/female'` for the `'SEX_LABEL'` column and `'Total'`, `'Minority'`, `'Nonminority'`, and `'Equally minority/nonminority'` for the `'RACE_GROUP_LABEL'` column.

In [26]:
df_company_summary_Overall = df_company_summary_Overall[df_company_summary_Overall["SEX_LABEL"] != "Total"]
df_company_summary_Overall = df_company_summary_Overall[df_company_summary_Overall["SEX_LABEL"] != "Equally male/female"]
df_company_summary_Overall = df_company_summary_Overall[df_company_summary_Overall["RACE_GROUP_LABEL"] != "Total"]
df_company_summary_Overall = df_company_summary_Overall[df_company_summary_Overall["RACE_GROUP_LABEL"] != "Minority"]
df_company_summary_Overall = df_company_summary_Overall[df_company_summary_Overall["RACE_GROUP_LABEL"] != "Nonminority"]
df_company_summary_Overall = df_company_summary_Overall[df_company_summary_Overall["RACE_GROUP_LABEL"] != "Equally minority/nonminority"]

5. Group the data by the categorical columns `'Location'`, `'SEX_LABEL'`, and `'RACE_GROUP_LABEL'` and aggragated the sum of the `'FIRMPDEMP'`. 

In [27]:
df_company_summary_Overall.groupby(["Location", "SEX_LABEL", "RACE_GROUP_LABEL" ] ,as_index=False)["FIRMPDEMP"].sum()

,Location,SEX_LABEL,RACE_GROUP_LABEL,FIRMPDEMP
0,United States,Female,American Indian and Alaska Native,8985
1,United States,Female,Asian,154984
2,United States,Female,Black or African American,43163
3,United States,Female,Native Hawaiian and Other Pacific Islander,0
4,United States,Female,White,945514
5,United States,Male,American Indian and Alaska Native,14559
6,United States,Male,Asian,336622
7,United States,Male,Black or African American,69624
8,United States,Male,Native Hawaiian and Other Pacific Islander,4468
9,United States,Male,White,3087049


6. Sort the values by the categorical columns `'Location'`, `'SEX_LABEL'`, and `'RACE_GROUP_LABEL'` in alphabetical order for ease of refrence when viewing in tabular form. 

In [28]:
df_company_summary_Overall = df_company_summary_Overall.sort_values(by=["Location","RACE_GROUP_LABEL", "SEX_LABEL"], ascending=True)

7. Reset the index and dropped the unneeded index column as it refrenced the original table rather than the new cleaned table `df_company_summary_Overall`.

In [29]:
df_company_summary_Overall.reset_index(level = 0, inplace= True)
df_company_summary_Overall = df_company_summary_Overall.drop("index", axis=1)
df_company_summary_Overall.head(10)

,Location,SEX_LABEL,RACE_GROUP_LABEL,FIRMPDEMP
0,United States,Female,American Indian and Alaska Native,8985
1,United States,Male,American Indian and Alaska Native,14559
2,United States,Female,Asian,154984
3,United States,Male,Asian,336622
4,United States,Female,Black or African American,43163
5,United States,Male,Black or African American,69624
6,United States,Female,Native Hawaiian and Other Pacific Islander,0
7,United States,Male,Native Hawaiian and Other Pacific Islander,4468
8,United States,Female,White,945514
9,United States,Male,White,3087049


This leaves the `df_company_summary_Overall DataFrame` with 4 columns for refrence. 

## **Charactaristics of Businesses**

### Calling the API
1. Call the following URL: http://api.census.gov/data/2018/abscb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,FIRMPDEMP,EMP,PAYANN&for=us:*&key={apikey}


In [30]:

response = requests.get(f'http://api.census.gov/data/2018/abscb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,FIRMPDEMP,EMP,PAYANN&for=us:*&key={apikey}')

2. Load the data from the API into a pandas DataFrame called ```business```. The column names load in as the first row in the DataFrame, ensure that the columns are named properly. After doing so, drop the first row of the DataFrame.

In [31]:
response_dict = response.json()
business = pd.DataFrame.from_dict(response_dict)
business.columns = business.iloc[0]
business = business.drop(business.index[0])

### Data Cleaning and Transforming

1. Drop the columns ```GEO_ID```, ```NAME, NAICS2017,SEX, ETH_GROUP, RACE_GROUP, VET_GROUP, us ```


In [32]:
drop_columns = ['GEO_ID','NAME','NAICS2017','SEX','ETH_GROUP','RACE_GROUP','us']
business = business.drop(drop_columns, axis=1)

2. Create more meaningful column names

|Original Column Name |New Column Name |
|--- | --- |
|`NAICS2017_LABEL`|`industry`|
|`SEX_LABEL`|`sex`|
|`ETH_GROUP_LABEL`|`ethnic_group`|
|`RACE_GROUP_LABEL`|`race`|
|`FIRMPDEMP`|`employer_firms`|
|`EMP`|`employees`|
|`PAYANN`|`payroll`|

In [33]:
business.rename(columns={'NAICS2017_LABEL':'industry',
                        'SEX_LABEL':'sex',
                        'ETH_GROUP_LABEL':'ethnic_group',
                        'RACE_GROUP_LABEL':'race',
                        'FIRMPDEMP':'employer_firms',
                        'EMP':'employees',
                        'PAYANN':'payroll'}, inplace=True)


4. Change the data types for all numeric columns to `float`.

In [34]:
columns_float = ['employer_firms', 'employees', 'payroll']

for column in columns_float:
    business[column]=business[column].astype('float')

### Making tables out of each demographic group

For each demographic column (`sex`, `ethnic_group`, `race`), make a table that contains information about the industry, number of employer firms, number of employees, and annual payroll.

1. Make variables to store each of these new DataFrames

|Demographic Column Name |New DataFrame Name |
|--- | --- |
|`sex`|`business_sex`|
|`ethnic_group`|`business_ethnicity`|
|`race`|`business_race`|

2. Start by simply storing a copy of the main `business` DataFrame

3. Only include the demograpic column and the `industry`,`employer_firms`,`revenue`,`employees`,and `payroll` columns in the new DataFrame. 
4. For each DataFrame, exclude the rows where the demographic label is `Total`, `Classifiable`, and `Unclassifiable`.
5. Group each DataFrame by `industry` and the demographic group while aggregating the numberic columns by their sum

In [35]:
business_sex = business.copy()
business_sex = business_sex[['industry','sex','employer_firms','employees','payroll']]
business_sex = business_sex[(business_sex['sex']=='Female')|(business_sex['sex']=='Male')]
business_sex=business_sex.groupby(['industry','sex'],as_index=False).sum()

In [36]:
business_ethnicity = business.copy()
business_ethnicity = business_ethnicity[['industry','ethnic_group','employer_firms','employees','payroll']]
business_ethnicity = business_ethnicity[(business_ethnicity['ethnic_group']!='Total')&(business_ethnicity['ethnic_group']!='Classifiable')&(business_ethnicity['ethnic_group']!='Unclassifiable')]
business_ethnicity=business_ethnicity.groupby(['industry','ethnic_group'],as_index=False).sum()

In [37]:
business_race = business.copy()
business_race = business_race[['industry','race','employer_firms','employees','payroll']]
business_race = business_race[(business_race['race']!='Total')&(business_race['race']!='Classifiable')&(business_race['race']!='Unclassifiable')]
business_race=business_race.groupby(['industry','race'],as_index=False).sum()

We now have 4 DataFrames we can use to make visualizations

## **Charactaristics of Business Owners**

### Calling the API
1. Create a national link that calls the desired data with a personal api key. Create a state link that calls the desired data with a personal api key. We analyzed demographic information on the national level, and business ownership rates on the state level so we called race and sex in the national link, and ownerhip numbers in the state link.

    For US Level Data
    https://api.census.gov/data/2018/abscbo?get=NAME,OWNER_SEX,OWNER_SEX_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNPDEMP&for=us:*&key={apikey}

    For State Level Data
    https://api.census.gov/data/2018/abscbo?get=NAME,OWNPDEMP&for=state:*&key={apikey}

In [38]:
nationalLink = f'https://api.census.gov/data/2018/abscbo?get=NAME,OWNER_SEX,OWNER_SEX_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNPDEMP&for=us:*&key={apikey}'
stateLink = f'https://api.census.gov/data/2018/abscbo?get=NAME,OWNPDEMP&for=state:*&key={apikey}'

*US Level*
1. Use `requests.get()` to call `nationalLink`
2. Create more meaningful column names

|Original Column Name |New Column Name |
|--- | --- |
|`NAME`|`location`|
|`OWNER_SEX`|`sex`|
|`OWNER_SEX_LABEL`|`sexLabel`|
|`OWNER_RACE`|`race`|
|`OWNER_RACE_LABEL`|`raceLabel`|
|`OWNPDEMP`|`numOwners`|
|`us`|`us`|

3. Load the API data into a pandas DataFrame called `businessOwner`


In [39]:
parameters = {}
r = requests.get(nationalLink, parameters)
colNames = ["location","sex", "sexLabel", "race", "raceLabel","numOwners","us"]
businessOwner = pd.DataFrame(columns=colNames, data=r.json()[1:])

4. Remove rows that contain aggrageted total data rather than data based on individual races or sexes (where `race=00` and `sex=001`)

In [40]:
businessOwner = businessOwner[businessOwner.sex.isin(['001']) == False]
businessOwner = businessOwner[businessOwner.race.isin(['00']) == False]

5. Drop uncessary columns such as `location` as we know we are looking at the whole U.S and the `sex` and `race` columns as we will only look at demographic labels.

In [41]:
businessOwner = businessOwner.drop(['location','sex', 'race','us'], axis=1)

6. Convert `numOnwers` to an `int` value.

In [42]:
businessOwner['numOwners'] = businessOwner['numOwners'].astype(int)

*State Level*
1. Use `requests.get()` to call `stateLink`
2. Create more meaningful column names

|Original Column Name |New Column Name |
|--- | --- |
|`NAME`|`State`|
|`OWNPDEMP`|`numOwners`|
|`state`|`state`|

3. Load the API data into a pandas DataFrame called `businessOwnerState`

In [43]:
parameters = {}
r = requests.get(stateLink, parameters)
colNames = ["State","numOwners","state"]
businessOwnerState = pd.DataFrame(columns=colNames, data=r.json()[1:])

4. Drop the `state` column

In [44]:
businessOwnerState = businessOwnerState.drop(['state'], axis=1)

5. Convert `numOnwers` to an `int` value.

In [45]:
businessOwnerState['numOwners'] = businessOwnerState['numOwners'].astype(int)

6. Merge `census_national_populations` DataFrame onto the `businessOwnersState` DataFrame using an inner join on their shared location column. This merged DataFrame is called `businessOwnersState`

In [46]:
businessOwnerState = businessOwnerState.merge(census_national_populations, how='inner', on='State')

7. Create a new column in the `businessOwnersState` DataFrame called `ownerPer100People` to show business owners per 100 people. Divide `numOwners` by population and multiply by 100.

In [47]:
businessOwnerState["onwerPer100People"] = (businessOwnerState["numOwners"] / businessOwnerState["Population"]) * 100

## **Technology charactaristics of Businesses**

### Calling the API

1. Call the following URLs

    For US level data
    https://api.census.gov/data/2018/abstcb?get=NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,NSFSZFI,NSFSZFI_LABEL,TECHUSE,TECHUSE_LABEL,FACTORS_P,FACTORS_P_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F&for=us:*&key={apikey}

    For State level data
    https://api.census.gov/data/2018/abstcb?get=NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,TECHUSE,FIRMPDEMP&for=state:*&key={apikey}


In [48]:

response_tech_summary_us = requests.get(f'https://api.census.gov/data/2018/abstcb?get=NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,NSFSZFI,NSFSZFI_LABEL,TECHUSE,TECHUSE_LABEL,FACTORS_P,FACTORS_P_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F&for=us:*&key={apikey}').json()

response_tech_summary_state = requests.get(f'https://api.census.gov/data/2018/abstcb?get=NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,TECHUSE,FIRMPDEMP&for=state:*&key={apikey}').json()

2. Load the data from the US level call into a pandas DataFrame called `response_tech_summary_us`. Load the data from the State level call into a pandas DataFrame called `response_tech_summary_state`.
3. The first row of both dataframes actually contain the column names. Ensure that the columns are named properly.

In [49]:
tech_summary_us  = pd.DataFrame(data=response_tech_summary_us[1:], columns = response_tech_summary_us[0])
tech_summary_state = pd.DataFrame(data=response_tech_summary_state[1:], columns = response_tech_summary_state[0])

4. We only want to keep rows that provide information that is useful for our visualizations. In the `tech_summary_us` DataFrame, only keep the `LABEL` and meaningful numeric columns. These are the columns that should be kept in the DataFrame: `NAME`, `NAICS2017_LABEL`, `SEX_LABEL`,`ETH_GROUP_LABEL`, `RACE_GROUP_LABEL`,`VET_GROUP_LABEL`, `TECHUSE`, `TECHUSE_LABEL`, `NSFSZFI_LABEL`,       `FACTORS_P_LABEL`,`FIRMPDEMP`, `FIRMPDEMP_F`,`RCPPDEMP`, `RCPPDEMP_F`, `EMP`, `EMP_F`, `PAYANN`,`PAYANN_F`.

In [50]:
tech_summary_us = tech_summary_us[['NAME', 'NAICS2017_LABEL', 'SEX_LABEL','ETH_GROUP_LABEL', 'RACE_GROUP_LABEL','VET_GROUP_LABEL', 'TECHUSE', 'TECHUSE_LABEL','NSFSZFI_LABEL',       'FACTORS_P_LABEL','FIRMPDEMP', 'FIRMPDEMP_F','RCPPDEMP', 'RCPPDEMP_F', 'EMP', 'EMP_F', 'PAYANN','PAYANN_F']]

5. Merge the `tech_summary_us` and `tech_summary_state` using an inner join on the `TECHUSE` column. This merged DataFrame is called `tech_summary_state`.


In [51]:
tech_summary_state = tech_summary_us.merge(tech_summary_state, how = 'inner', on = 'TECHUSE')

6. Only keep the following columns in the `tech_summary_state` DataFrame: `NAME_y`, `NAICS2017_LABEL_y`, `TECHUSE`,`TECHUSE_LABEL`, `FIRMPDEMP_y`.

In [52]:
tech_summary_state = tech_summary_state[['NAME_y', 'NAICS2017_LABEL_y', 'TECHUSE','TECHUSE_LABEL', 'FIRMPDEMP_y']]

7. Rename the columns to more meaningful names.

|Original Column Name |New Column Name |
|--- | --- |
|`NAME_y`|`State`|
|`NAICS2017_LABEL_y`|`NAICS2017_LABEL`|
|`FIRMPDEMP_y`|`FIRMPDEMP`|


In [53]:
tech_summary_state.rename(columns={"NAME_y":"State","NAICS2017_LABEL_y":"NAICS2017_LABEL","FIRMPDEMP_y":"FIRMPDEMP"},inplace=True )

8. Ensure all numeric columns are integers

In [54]:
tech_summary_state['FIRMPDEMP'] = tech_summary_state['FIRMPDEMP'].apply(lambda x: int(x))
tech_summary_us['FIRMPDEMP'] = tech_summary_us['FIRMPDEMP'].apply(lambda x: int(x))
tech_summary_us['RCPPDEMP'] = tech_summary_us['RCPPDEMP'].apply(lambda x: int(x))
tech_summary_us['EMP'] = tech_summary_us['EMP'].apply(lambda x: int(x))
tech_summary_us['PAYANN'] = tech_summary_us['PAYANN'].apply(lambda x: int(x))

9. Merge the `tech_summary_state` and the `census_national_populations` with an inner merge on the `State` column. Call this merged DataFrame `tech_summary_state`.

In [55]:
tech_summary_state = tech_summary_state.merge(census_national_populations, how = 'inner', on = 'State')

10. Create a column on the `tech_summary_state` DataFrame called `FIRMTOPOP`. Calculate the number of firms per 100 people by dividing `FRIMPDEMP` BY `Population` and multiplying by 100.

In [56]:
tech_summary_state['FIRMTOPOP'] = tech_summary_state['FIRMPDEMP']/tech_summary_state['Population']*100

11. Drop any duplicates form the `tech_summary_state` and `tech_summary_us` DataFrames that may have come from the merge

In [57]:
tech_summary_state.drop_duplicates(inplace = True)
tech_summary_us.drop_duplicates(inplace= True)

## Conclusion
---

After pulling in data from the Census Bureau's API, loading in some extra CSVs, and transforming multiple tables, we are now able to effectively create some visualizations.